In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Dresden,DE,2020-06-24 22:21:37,51.05,13.74,62.60,82,87,2.24
1,1,Port Elizabeth,ZA,2020-06-24 22:14:37,-33.92,25.57,53.60,50,0,5.82
2,2,Rikitea,PF,2020-06-24 22:15:00,-23.12,-134.97,70.59,55,97,9.80
3,3,Kushima,JP,2020-06-24 22:22:26,31.46,131.23,75.20,88,90,2.24
4,4,Cape Town,ZA,2020-06-24 22:15:01,-33.93,18.42,60.01,67,1,11.41


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5
                  )
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4) 
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:

# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"] 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=50, point_radius=4
                                )

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Kushima,JP,2020-06-24 22:22:26,31.46,131.23,75.20,88,90,2.24
5,5,Albany,US,2020-06-24 22:13:50,42.60,-73.97,84.00,33,1,10.07
9,9,Washington,US,2020-06-24 22:19:09,47.50,-120.50,88.00,27,0,15.99
12,12,Tabuk,PH,2020-06-24 22:22:27,17.42,121.44,75.96,87,100,2.77
20,20,Awbari,LY,2020-06-24 22:22:29,26.59,12.78,85.30,17,0,9.06
21,21,Corn Island,NI,2020-06-24 22:22:29,12.17,-83.04,82.36,79,98,21.50
23,23,Bethel,US,2020-06-24 22:14:03,41.37,-73.41,84.99,34,1,13.87
26,26,Pecos,US,2020-06-24 22:22:30,31.42,-103.49,89.60,35,1,13.87
28,28,Sao Filipe,CV,2020-06-24 22:13:53,14.90,-24.50,75.78,84,28,5.95
31,31,Russell,US,2020-06-24 22:13:36,32.35,-85.20,80.60,88,40,8.99


In [11]:
preferred_cities_df.count()

City_ID       197
City          197
Country       197
Date          197
Lat           197
Lng           197
Max Temp      197
Humidity      197
Cloudiness    197
Wind Speed    197
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Kushima,JP,75.20,31.46,131.23,
5,Albany,US,84.00,42.60,-73.97,
9,Washington,US,88.00,47.50,-120.50,
12,Tabuk,PH,75.96,17.42,121.44,
20,Awbari,LY,85.30,26.59,12.78,
21,Corn Island,NI,82.36,12.17,-83.04,
23,Bethel,US,84.99,41.37,-73.41,
26,Pecos,US,89.60,31.42,-103.49,
28,Sao Filipe,CV,75.78,14.90,-24.50,
31,Russell,US,80.60,32.35,-85.20,


In [13]:
# Set parameters to search for a hotel.
params = {
    "radius": 20000,
    "type": "lodging",
    "key": g_key
}

In [14]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 30000,
    "types": "lodging",
    "key": g_key,
    "location": "70.63, 147.92"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()


In [15]:
hotels


{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}

In [16]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
# Grab the first hotel from the results and store the name.
# Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Kushima,JP,75.20,31.46,131.23,Shibushiwan Daikoku Resort Hotel
5,Albany,US,84.00,42.60,-73.97,CrestHill Suites SUNY University Albany
9,Washington,US,88.00,47.50,-120.50,FairBridge Inn & Suites Leavenworth
12,Tabuk,PH,75.96,17.42,121.44,Grand Zion Garden Resort Hotel
20,Awbari,LY,85.30,26.59,12.78,فندق أوباري


In [18]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [19]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [20]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))